Consider two tasks: check if running shoes from RunPro 3000 do make a difference (50-50 means as total average the shoes should be normalised, to be checked if that is also true per attribute).

Attributes are the following:
* age group
* country
* male or female

Second task: try to predict the finishing time of the Marathon as part of the attributes.

So technically task 1 is a classification problem, task 2 is a regression problem